In [1]:
import math

from solid import *
from solid.utils import *

from subprocess import run

import viewscad
r = viewscad.Renderer()

In [2]:
import os, solid;
print(os.path.dirname(solid.__file__) + '/examples')

/home/flo/.local/lib/python3.9/site-packages/solid/examples


In [3]:
# CONFIGURATION
GENERATE_WITH_KEYCAPS = True
HOLE_HEIGHT_TOLERANCE = 0.2 # 0 gives weird viewing glitches

In [32]:
KEYCAP_SIZE = 18
KEYCAP_HEIGHT = 9
KEYCAP_Z_OFFSET = 6.6

KNOB_RADIUS = 7
KNOB_HEIGHT = 16
KNOB_Z_OFFSET = 4

PLATE_GRID = 19.05
PLATE_HOLE = 14
PLATE_THICKNESS = 4
# PLATE_SPACER = SWITCH_GRID - PLATE_HOLE  # = 5.05mm

def plate_cutout(units_x=1, units_y=1, withCap=False, encoder=False, center=True):
    if units_x == 0 or units_y == 0:
        withCap = False
        if units_x == 0:
            units_x = 1
            units_y = 1
            cutout = cube([PLATE_GRID,PLATE_GRID,PLATE_THICKNESS], center=True)
        if units_y == 0:
            units_y = 1
            units_x = 1
            cutout = cube(0)
    else:
        plate = cube([PLATE_GRID*units_x,PLATE_GRID*units_y,PLATE_THICKNESS], center=True)
        hole = cube([PLATE_HOLE,PLATE_HOLE,PLATE_THICKNESS+HOLE_HEIGHT_TOLERANCE], center=True)
        if encoder:
            width = 8.37/2 # mm, from the center
            height = PLATE_HOLE/2 + 1.7 # mm, from the center
            filler_points = [
                [-width,height,0],
                [width,height,0],
                [width,-height,0],
                [-width,-height,0],
                [-width,PLATE_HOLE/2,PLATE_THICKNESS*4/5],
                [width,PLATE_HOLE/2,PLATE_THICKNESS*4/5],
                [width,-PLATE_HOLE/2,PLATE_THICKNESS*4/5],
                [-width,-PLATE_HOLE/2,PLATE_THICKNESS*4/5]
            ]
            filler_faces = [
                [3,2,1,0],
                [4,5,6,7],
                [0,1,5,4],
                [1,2,6,5],
                [2,3,7,6],
                [3,0,4,7],
            ]
            encoder_cutout = polyhedron(points=filler_points,faces=filler_faces,convexity=5)
            hole += down(PLATE_THICKNESS/2)(encoder_cutout)
        cutout = plate - hole
    
    # TODO: if units_x or units_y >= 2: add stab holes

    if withCap:
        if encoder:
            knob = color([0.0,1.0,0.5])(cylinder(r=KNOB_RADIUS, h=KNOB_HEIGHT, center=True))
            knob = up(PLATE_THICKNESS/2 + KNOB_HEIGHT/2 + KNOB_Z_OFFSET)(knob)
            cutout += knob
        else:
            spacing = (PLATE_GRID - KEYCAP_SIZE)
            cap = color([0.0,1.0,0.5])(cube([PLATE_GRID*units_x - spacing,PLATE_GRID*units_y - spacing,KEYCAP_HEIGHT], center=True))
            cap = up(PLATE_THICKNESS/2 + KEYCAP_HEIGHT/2 + KEYCAP_Z_OFFSET)(cap)
            cutout += cap
        
    
    if(center == False):
        cutout = left(units_x*PLATE_GRID/2)(back(units_y*PLATE_GRID/2)(cutout))
    
    return cutout

r.render(plate_cutout(units_x=1, units_y=1, withCap=True, encoder=True, center=True))

In [55]:
# keys[units_y], units_x, center, encoder[]
def plate_column(keys=[1,1,1,1], units_x=1, center=True, encoder=[]):
    if len(encoder) == 0 or len(encoder) != len(keys):
        encoder = [0] * len(keys)
    i,offset,column = 0,0,cube(0)
    for units_y in keys:
        cutout = plate_cutout(units_x=units_x, units_y=units_y, withCap=GENERATE_WITH_KEYCAPS, encoder=encoder[i])
        if units_y == 0:
            units_y = 1
        cur_offset = units_y/2
        column += forward(PLATE_GRID*(offset + cur_offset))(cutout)
        offset += (units_y/2) + cur_offset
        i += 1
    if(center == False):
        column = right(units_x*PLATE_GRID/2)(column)
    return column
        
r.render(plate_column(keys=[2,0,2,1], units_x=1.5, center=False, encoder=[0,0,0,1]))

In [56]:
# [column[keys[],units,y_offset_units,encoder[]]]
# keep the y_offset_units in multiple or halfs of 1u -> for easier alignment in KiCAD later

def plate(layout):
    i,cur_offset,offset,plate = 0,0,0,cube(0)
    for column_specs in layout:
        if len(column_specs) == 4:
            encoder = column_specs[3]
        else:
            encoder = []
        column = plate_column(keys=column_specs[0], units_x=column_specs[1], center=True, encoder=encoder)
        cur_offset = column_specs[1]/2
        plate += right(PLATE_GRID*(offset+cur_offset))(forward(PLATE_GRID*column_specs[2])(column))
        offset += column_specs[1]/2 + cur_offset
        i += 1
    return plate
    

r.render(plate([[[2,1,1.5,1],1.5,0],[[1,1,1,1.5],1,0.5],[[1,1,1,1,2],2,-0.5],[[1,2,1],1,0,[1,1,1]]]))

In [58]:
C4 = [1,1,1,1]
C5 = [1,1,1,1,1]
C3_vert = [1.5,1.5,1]
C4_ortho = [0,0,0,1]

E5_enc = [1,0,0,0,0]

def myPlate(layout=[C4,1,0], space_angle_degrees=12, space_units=1.5, thumbcluster_keys=[1,1], ortho=False):
    myPlate = plate(layout)

    space_angle = math.radians(space_angle_degrees) # TODO den nicesten Winkel finden ( ͡° ͜ʖ ͡°)

    # predefined by my layout:
    space_offset_columns = 5

    space_offset_y = layout[space_offset_columns][2] - layout[space_offset_columns-1][2]
    filler_faces = [[0,2,1],[0,1,4,3],[1,2,5,4],[2,0,3,5],[3,4,5]]

    i,plate_x,space_offset_available = 0,0,0 # plate x axis length; amount of units left after the offset
    for column in layout:
        if i >= space_offset_columns:
            space_offset_available += column[1]
        plate_x += column[1]
        i += 1
    
    space = plate_cutout(units_x=space_units, center=False, withCap=GENERATE_WITH_KEYCAPS)
    space = right(PLATE_GRID*space_units)(space) # set anchor on top right edge
    thumbcluster = plate_column(keys=thumbcluster_keys, units_x=1, center=False)

    if(space_angle >= math.pi/4):
        # idk why I calculated that one as well
        adjacent = math.cos(math.pi/2-space_angle)*PLATE_GRID
        opposite = math.sin(math.pi/2-space_angle)*PLATE_GRID
        # this probably doesn't work as expected. I realised too late that this isn't even the angle I wanted
        space_edge_y = opposite + space_units*PLATE_GRID/math.sin(space_angle)
        space_edge_x = (space_edge_y-opposite)/math.tan(space_angle)
    else:
        # this is the working and tested version of this code
        adjacent = space_offset_y*PLATE_GRID/math.tan(math.pi/2-space_angle)
        opposite = space_offset_y*PLATE_GRID
        hypotenuse =  math.hypot(adjacent, opposite)

        opposite_offset = (PLATE_GRID-hypotenuse)/math.cos(space_angle)
        adjacent_offset = (PLATE_GRID-hypotenuse)*math.sin(space_angle)

        space_edge_y = opposite + opposite_offset + space_units*PLATE_GRID*math.sin(space_angle)
        space_edge_x = space_units*PLATE_GRID*math.cos(space_angle) - adjacent_offset


    # position the thumbcluster
    angle_space_thumbcluster = math.atan((space_offset_available*PLATE_GRID - space_edge_x)/space_edge_y) - space_angle
    if angle_space_thumbcluster < 0:
        angle_space_thumbcluster = 0

    mini_spacing = PLATE_GRID*math.tan(angle_space_thumbcluster) # parallel distance from the top right edge of the space key to the thumbcluster
    mini_spacing_hypotenuse = PLATE_GRID/math.cos(angle_space_thumbcluster)
    filler_points = [
        [0,0,-PLATE_THICKNESS/2],
        [mini_spacing,PLATE_GRID,-PLATE_THICKNESS/2],
        [0,PLATE_GRID,-PLATE_THICKNESS/2],
        [0,0,PLATE_THICKNESS/2],
        [mini_spacing,PLATE_GRID,PLATE_THICKNESS/2],
        [0,PLATE_GRID,PLATE_THICKNESS/2]
    ]
    filler = polyhedron(points=filler_points,faces=filler_faces,convexity=5) # between space and thumb keys
    space += back(PLATE_GRID)(right(space_units*PLATE_GRID)(filler))

    thumbcluster = rotate(a=math.degrees(angle_space_thumbcluster), v=DOWN_VEC)(thumbcluster) # rotate
    thumbcluster = back(PLATE_GRID)(right(space_units*PLATE_GRID)(thumbcluster)) # move next to space
    space += thumbcluster

    filler_points = [
        [0,0,-PLATE_THICKNESS/2],
        [adjacent,0,-PLATE_THICKNESS/2],
        [0,-opposite,-PLATE_THICKNESS/2],
        [0,0,PLATE_THICKNESS/2],
        [adjacent,0,PLATE_THICKNESS/2],
        [0,-opposite,PLATE_THICKNESS/2]
    ]
    filler = polyhedron(points=filler_points,faces=filler_faces,convexity=5) # between space and column to the left

    if ortho == True:
        pfusch = 1 # mm
        thumbkey = plate_cutout(units_x=1, units_y=1, center=False, withCap=GENERATE_WITH_KEYCAPS, encoder=True) # space is too tight for a cap
        thumbkey = right(mini_spacing + space_units*PLATE_GRID + pfusch)(forward(PLATE_GRID)(thumbkey))
        space += thumbkey
    
    space = rotate(a=math.degrees(space_angle), v=DOWN_VEC)(space) # rotate

    space_offset_y = layout[space_offset_columns][2]
    space_offset_x = plate_x - space_offset_available
    
    myPlate += right(space_offset_x*PLATE_GRID)(forward(space_offset_y*PLATE_GRID)(filler))
    myPlate += right(space_offset_x*PLATE_GRID + adjacent)(forward(space_offset_y*PLATE_GRID)(space))

    filler_x = (space_units*PLATE_GRID+mini_spacing)*math.cos(space_angle) + adjacent
    filler_y = -(space_units*PLATE_GRID+mini_spacing)*math.sin(space_angle)
    mini_spacing2 = math.hypot(filler_x - space_offset_available*PLATE_GRID, filler_y - 0) # length of the opposite of the current filler
    if ortho == False:
        filler_points = [
            [adjacent,0,-PLATE_THICKNESS/2],
            [filler_x,filler_y,-PLATE_THICKNESS/2],
            [space_offset_available*PLATE_GRID,0,-PLATE_THICKNESS/2],
            [adjacent,0,PLATE_THICKNESS/2],
            [filler_x,filler_y,PLATE_THICKNESS/2],
            [space_offset_available*PLATE_GRID,0,PLATE_THICKNESS/2]
        ]
        filler = polyhedron(points=filler_points,faces=filler_faces,convexity=5) # between space and the two columns above
        myPlate += right(space_offset_x*PLATE_GRID)(forward(space_offset_y*PLATE_GRID)(filler))

    # get size of thumb cluster
    i,thumbcluster_length = 0,0
    while(i < len(thumbcluster_keys)):
        thumbcluster_length += thumbcluster_keys[i]*PLATE_GRID
        i += 1
    adjacent = thumbcluster_length - mini_spacing2 - mini_spacing_hypotenuse
    filler_x = math.sin(space_angle + angle_space_thumbcluster)*adjacent # adjacent is the hypotenuse in this case
    filler_y = math.cos(space_angle + angle_space_thumbcluster)*adjacent # adjacent is the hypotenuse in this case
    hypotenuse = adjacent/math.cos(space_angle + angle_space_thumbcluster)
    if ortho == False:
        filler_points = [
            [0,0,-PLATE_THICKNESS/2],
            [filler_x,filler_y,-PLATE_THICKNESS/2],
            [0,hypotenuse,-PLATE_THICKNESS/2],
            [0,0,PLATE_THICKNESS/2],
            [filler_x,filler_y,PLATE_THICKNESS/2],
            [0,hypotenuse,PLATE_THICKNESS/2],
        ]
        filler = polyhedron(points=filler_points,faces=filler_faces,convexity=5) # between thumb keys and the grid column to the left
        myPlate += right(plate_x*PLATE_GRID)(forward(space_offset_y*PLATE_GRID)(filler))
    
    return myPlate

r.render(myPlate(
    layout=[[C4,1.5,0],[C4,1,0],[C5,1,-0.5,E5_enc],[C5,1,-0.375],[C5,1,-0.5],[C4,1,0.25],[C4_ortho,1,0.25]],
    space_angle_degrees=12,
    space_units=1.5,
    ortho=True
))

In [8]:
layouts = [
    [[C4,1.5,0],[C4,1,0],[C5,1,-0.5],[C5,1,-0.375],[C5,1,-0.5],[C4,1,0.25],[C3_vert,1,0.25]], # ergo caps
    [[C4,1,0],[C4,1,0],[C5,1,-0.5],[C5,1,-0.375],[C5,1,-0.5],[C4,1,0.25],[C4_ortho,1,0.25]] # ortho caps (+OLED)
]
angle = 12
space_keys = [
    1.5, # normal sizing
    1.75 # 40% sizing
]

# r.render(myPlate(layout=layouts[0],space_angle_degrees=12,space_units=space_keys[0]))
# scad_render_to_file(myPlate(layout=layouts[0],space_angle_degrees=12,space_units=space_keys[0]), 'plate_0.scad')
# r.render(myPlate(layout=layouts[0],space_angle_degrees=12,space_units=space_keys[1]))
# scad_render_to_file(myPlate(layout=layouts[0],space_angle_degrees=12,space_units=space_keys[1]), 'plate_1.scad')
# r.render(myPlate(layout=layouts[1],space_angle_degrees=12,space_units=space_keys[0]))
# r.render(myPlate(layout=layouts[1],space_angle_degrees=12,space_units=space_keys[1]))

In [9]:
def getSVG(object, name="projection"):
    scad_render_to_file(projection(cut=True)(object), name + ".scad")
    run(["openscad", "-o",  name + ".svg", name + ".scad"])

# getSVG(myP, "projection2")

In [10]:
# generate valid openscad code and store it in file
scad_render_to_file(myPlate(layout=layouts[0],space_angle_degrees=12,space_units=space_keys[0]), 'plate.scad')

# run openscad and export to stl
run(["openscad", "-o",  "plate.stl", "plate.scad"])

CompletedProcess(args=['openscad', '-o', 'plate.stl', 'plate.scad'], returncode=0)

In [11]:
#! /usr/bin/env python3
from solid.objects import linear_extrude
from solid.solidpython import OpenSCADObject
import sys
from math import cos, radians, sin, pi, tau
from pathlib import Path

from euclid3 import Point2, Point3, Vector3

from solid import scad_render_to_file, text, translate, cube, color, rotate
from solid.utils import UP_VEC, Vector23, distribute_in_grid, extrude_along_path
from solid.utils import down, right, frange, lerp


from typing import Set, Sequence, List, Callable, Optional, Union, Iterable, Tuple

SEGMENTS = 48
PATH_RAD = 50 
SHAPE_RAD = 15

TEXT_LOC = [-0.6 *PATH_RAD, 1.6 * PATH_RAD]

def basic_extrude_example():
    path_rad = PATH_RAD
    shape = star(num_points=5)
    path = sinusoidal_ring(rad=path_rad, segments=240)

    # At its simplest, just sweep a shape along a path
    extruded = extrude_along_path( shape_pts=shape, path_pts=path)
    extruded += make_label('Basic Extrude')
    return extruded
        
def extrude_example_xy_scaling() -> OpenSCADObject:
    num_points = SEGMENTS
    path_rad = PATH_RAD
    circle = circle_points(15)
    path = circle_points(rad = path_rad)

    # If scales aren't included, they'll default to
    # no scaling at each step along path.
    no_scale_obj = make_label('No Scale')
    no_scale_obj += extrude_along_path(circle, path)

    # angles: from 0 to 6*Pi
    angles = list((frange(0, 3*tau, num_steps=len(path))))

    # With a 1-D scale factor, an extrusion grows and shrinks uniformly
    x_scales = [(1 + cos(a)/2) for a in angles]
    x_obj = make_label('1D Scale')
    x_obj += extrude_along_path(circle, path, scales=x_scales)

    # With a 2D scale factor, a shape's X & Y dimensions can scale 
    # independently, leading to more interesting shapes
    # X & Y scales vary between 0.5 & 1.5
    xy_scales = [Point2( 1 + cos(a)/2, 1 + sin(a)/2) for a in angles]
    xy_obj = make_label('2D Scale')
    xy_obj += extrude_along_path(circle, path, scales=xy_scales)

    obj = no_scale_obj + right(3*path_rad)(x_obj) + right(6 * path_rad)(xy_obj)
    return obj

def extrude_example_capped_ends() -> OpenSCADObject:
    num_points = SEGMENTS/2
    path_rad = 50
    circle = star(6)
    path = circle_points(rad = path_rad)[:-4]

    # If `connect_ends` is False or unspecified, ends will be capped.
    # Endcaps will be correct for most convex or mildly concave (e.g. stars) cross sections
    capped_obj = make_label('Capped Ends')
    capped_obj += extrude_along_path(circle, path, connect_ends=False, cap_ends=True)

    # If `connect_ends` is specified, create a continuous manifold object
    connected_obj = make_label('Connected Ends')
    connected_obj += extrude_along_path(circle, path, connect_ends=True)   

    return capped_obj + right(3*path_rad)(connected_obj)

def extrude_example_rotations() -> OpenSCADObject:
    path_rad = PATH_RAD
    shape = star(num_points=5)
    path = circle_points(path_rad, num_points=240)

    # For a simple example, make one complete revolution by the end of the extrusion
    simple_rot = make_label('Simple Rotation')
    simple_rot += extrude_along_path(shape, path, rotations=[360], connect_ends=True)

    # For a more complex set of rotations, add a rotation degree for each point in path
    complex_rotations = []
    degs = 0
    oscillation_max = 60

    for i in frange(0, 1, num_steps=len(path)):
        # For the first third of the path, do one complete rotation
        if i <= 0.333:
            degs = i/0.333*360
        # For the second third of the path, oscillate between +/- oscillation_max degrees 
        elif i <= 0.666:
            angle = lerp(i, 0.333, 0.666, 0, 2*tau)
            degs = oscillation_max * sin(angle)
        # For the last third of the path, oscillate increasingly fast but with smaller magnitude 
        else:
            # angle increases in a nonlinear curve, so
            # oscillations should get quicker and quicker
            x = lerp(i, 0.666, 1.0, 0, 2)
            angle = pow(x, 2.2) * tau
            # decrease the size of the oscillations by a factor of 10
            # over the course of this stretch
            osc = lerp(i, 0.666, 1.0, oscillation_max, oscillation_max/10)
            degs = osc * sin(angle)
        complex_rotations.append(degs)

    complex_rot = make_label('Complex Rotation')
    complex_rot += extrude_along_path(shape, path, rotations=complex_rotations)

    # Make some red markers to show the boundaries between the three sections of this path
    marker_w = SHAPE_RAD * 1.5
    marker = translate([path_rad, 0, 0])(
        cube([marker_w, 1, marker_w], center=True)
    )
    markers = [color('red')(rotate([0,0,120*i])(marker)) for i in range(3)]
    complex_rot += markers

    return simple_rot + right(3*path_rad)(complex_rot)

def extrude_example_transforms() -> OpenSCADObject:
    path_rad = PATH_RAD
    height = 2*SHAPE_RAD
    num_steps = 120

    shape = circle_points(rad=path_rad, num_points=120)
    path = [Point3(0,0,i) for i in frange(0, height, num_steps=num_steps)]

    max_rotation = radians(15)
    max_z_displacement = height/10
    up = Vector3(0,0,1)

    # The transforms argument is powerful. 
    # Each point in the entire extrusion will call this function with unique arguments: 
    #   -- `path_norm` in [0, 1] specifying how far along in the extrusion a point's loop is
    #   -- `loop_norm` in [0, 1] specifying where in its loop a point is.
    def point_trans(point: Point3, path_norm:float, loop_norm: float) -> Point3:
        # scale the point from 1x to 2x in the course of the 
        # extrusion, 
        scale = 1 + path_norm*path_norm/2
        p = scale * point

        # Rotate the points sinusoidally up to max_rotation
        p = p.rotate_around(up, max_rotation*sin(tau*path_norm))


        # Oscillate z values sinusoidally, growing from 
        # 0 magnitude to max_z_displacement
        max_z = lerp(path_norm, 0, 1, 0, max_z_displacement)
        angle = lerp(loop_norm, 0, 1, 0, 10*tau)
        p.z += max_z*sin(angle)
        return p

    no_trans = make_label('No Transform')
    no_trans += down(height/2)(
        extrude_along_path(shape, path, cap_ends=False)
    )

    # We can pass transforms a single function that will be called on all points,
    # or pass a list with a transform function for each point along path
    arb_trans = make_label('Arbitrary Transform')
    arb_trans += down(height/2)(
        extrude_along_path(shape, path, transforms=[point_trans], cap_ends=False)
    )

    return no_trans + right(3*path_rad)(arb_trans)

# ============
# = GEOMETRY =
# ============
def sinusoidal_ring(rad=25, segments=SEGMENTS) -> List[Point3]:
    outline = []
    for i in range(segments):
        angle = radians(i * 360 / segments)
        scaled_rad = (1 + 0.18*cos(angle*5)) * rad
        x = scaled_rad * cos(angle)
        y = scaled_rad * sin(angle)
        z = 0
        # Or stir it up and add an oscillation in z as well
        # z = 3 * sin(angle * 6)
        outline.append(Point3(x, y, z))
    return outline

def star(num_points=5, outer_rad=SHAPE_RAD, dip_factor=0.5) -> List[Point3]:
    star_pts = []
    for i in range(2 * num_points):
        rad = outer_rad - i % 2 * dip_factor * outer_rad
        angle = radians(360 / (2 * num_points) * i)
        star_pts.append(Point3(rad * cos(angle), rad * sin(angle), 0))
    return star_pts

def circle_points(rad: float = SHAPE_RAD, num_points: int = SEGMENTS) -> List[Point2]:
    angles = frange(0, tau, num_steps=num_points, include_end=True)
    points = list([Point2(rad*cos(a), rad*sin(a)) for a in angles])
    return points

def make_label(message:str, text_loc:Tuple[float, float]=TEXT_LOC, height=5) -> OpenSCADObject:
    return translate(text_loc)(
        linear_extrude(height)(
            text(message)
        )
    )

# ===============
# = ENTRY POINT =
# ===============
if __name__ == "__main__":
    out_dir = ""

    basic_extrude = basic_extrude_example()
    scaled_extrusions = extrude_example_xy_scaling()
    capped_extrusions = extrude_example_capped_ends()
    rotated_extrusions = extrude_example_rotations()
    arbitrary_transforms = extrude_example_transforms()
    all_objs = [basic_extrude, scaled_extrusions, capped_extrusions, rotated_extrusions, arbitrary_transforms]

    a = distribute_in_grid(all_objs, 
                            max_bounding_box=[4*PATH_RAD, 4*PATH_RAD],
                            rows_and_cols=[len(all_objs), 1])

    file_out = scad_render_to_file(a,  out_dir=out_dir, include_orig_code=True)

I should have probably gone with a vectore based approach…